In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import paper_utils as pu

pu.set_plots()

# Load data

In [ ]:
data = {project: pu.load(project) for project in pu.PROJECTS}

# Train models and Figures S4-6

In [ ]:
temp_models = {}
qvals = [0.1, 0.25, 0.5, 0.75, 0.9]
qvalsmean = [f"{q}-mean" for q in qvals]
qvalsmax = [f"{q}-max" for q in qvals]
qvalsh6 = [f"{q}-h6" for q in qvals]
qvalsh9 = [f"{q}-h9" for q in qvals]
qvalsh12 = [f"{q}-h12" for q in qvals]
qvalsh15 = [f"{q}-h15" for q in qvals]
qvalsh16 = [f"{q}-h16" for q in qvals]
qvalsh20 = [f"{q}-h20" for q in qvals]

# We are saving three of these
figure_names = ["Figure S4.pdf", "Figure S5.pdf", "Figure S6.pdf"]
ifig = 0

for q in (
    qvalsmean + qvalsmax + qvalsh6 + qvalsh9 + qvalsh12 + qvalsh15 + qvalsh16 + qvalsh20
):
    with_plots = (
        # (q == "0.5-h20") or (q == "0.5-h6") or (q == "0.5-h15") or (q == "0.5-h12")
        (q == "0.5-h15")
        or (q == "0.75-h15")
        or (q == "0.9-h15")
        # or (q == "0.9-h6")
    )
    temp_models[q] = {}
    if with_plots:
        f, ax = plt.subplots(
            2, 3, figsize=(pu.PAGE_WIDTH, 2 * pu.ROW_HEIGHT), sharey=True, sharex=True
        )
        ax = ax.flatten()
    for i, project in enumerate(pu.PROJECTS):
        weekend = pu.choose_weekend[project]
        ylabel = f"Zone temperatures ($\degree$F)\n{q.split('-')[1]} {float(q.split('-')[0])*100:.0f}\u1D57\u02B0 percentile"
        if with_plots:
            pu.plot_base(
                data[project], y=q, ylabel=ylabel, fax=(f, ax[i]), weekend=weekend
            )
            fax = (f, ax[i])
        else:
            fax = None
        temp_models[q][project] = pu.create_model(
            data[project], fax=fax, weekend=weekend, y=q, log=False
        )
        if with_plots:
            ax[i].set_title(project)

    if with_plots:
        for a in ax:
            a.yaxis.set_tick_params(labelleft=True)
            a.xaxis.set_tick_params(labelbottom=True)
            a.set_ylim([71, 80])
        f.tight_layout()
        pu.add_legend((f, ax))
        if pu.save_fig:
            f.savefig(pu.si_path / figure_names[ifig])
            ifig += 1

# Base case - main results figure
Sanity check, this is not saved.

In [ ]:
color = [pu.grey1, pu.grey2, pu.dark_grey, pu.grey2, pu.grey1]
ms = [1, 1, 2, 1, 1]

f, ax = plt.subplots(1, 2, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 0.75))
ax = ax.flatten()

j = -0.2
for iq, q in enumerate(qvalsmean):
    for i, project in enumerate(pu.PROJECTS):
        pu.ci_plot(
            ax[0],
            i + j,
            temp_models[q][project],
            "Treated",
            color=color[iq],
            ms=ms[iq],
            log=False,
        )
        pu.ci_plot(
            ax[1],
            i + j,
            temp_models[q][project],
            "Tmean",
            color=color[iq],
            ms=ms[iq],
            log=False,
        )
    j += 0.1


for a in ax:
    a.set_xticks(range(len(pu.PROJECTS)))
    a.set_xticklabels(pu.PROJECTS, rotation=30, ha="right")
ax[0].set_ylabel("Impact ($\degree$F)")
ax[1].set_ylabel("Impact ($\degree$F/$\degree$F)")

ax[0].set_title(
    "Impact of 2$\degree$F setpoint increase on zone temp. percentiles\n(10\u1D57\u02B0, 25\u1D57\u02B0, 50\u1D57\u02B0, 75\u1D57\u02B0, 90\u1D57\u02B0)",
    ha="left",
    loc="left",
)
ax[1].set_title(
    "Impact of mean daily OAT on zone temp. percentiles\n(10\u1D57\u02B0, 25\u1D57\u02B0, 50\u1D57\u02B0, 75\u1D57\u02B0, 90\u1D57\u02B0)",
    ha="left",
    loc="left",
)

f.tight_layout(w_pad=2.0)
plt.subplots_adjust(top=0.8, bottom=0.3)

# Additional cases - Figure S7

In [ ]:
cases = {
    "6AM": qvalsh6,
    "12PM": qvalsh12,
    "3PM": qvalsh15,
    "8PM": qvalsh20,
    "max": qvalsmax,
}

for icase, case in enumerate(cases):
    f, ax = plt.subplots(1, 2, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 0.75))
    ax = ax.flatten()

    j = -0.2
    for iq, q in enumerate(cases[case]):
        for i, project in enumerate(pu.PROJECTS):
            pu.ci_plot(
                ax[0],
                i + j,
                temp_models[q][project],
                "Treated",
                color=color[iq],
                ms=ms[iq],
                log=False,
            )
            pu.ci_plot(
                ax[1],
                i + j,
                temp_models[q][project],
                "Tmean",
                color=color[iq],
                ms=ms[iq],
                log=False,
            )
        j += 0.1

    for a in ax:
        a.set_xticks(range(len(pu.PROJECTS)))
        a.set_xticklabels(pu.PROJECTS, rotation=30, ha="right")
    ax[0].set_ylabel("Impact ($\degree$F)")
    ax[1].set_ylabel("Impact ($\degree$F/$\degree$F)")
    ax[0].set_ylim(top=1.8)

    title0 = (
        f"{pu.plot_numbers[2*icase]}) "
        + f"Impact of 2$\degree$F setpoint increase on {case} zone temp.\npercentiles(10\u1D57\u02B0, 25\u1D57\u02B0, 50\u1D57\u02B0, 75\u1D57\u02B0, 90\u1D57\u02B0)"
    )
    title1 = (
        f"{pu.plot_numbers[2*icase+1]}) "
        + f"Impact of mean daily OAT on {case} zone temp.\npercentiles (10\u1D57\u02B0, 25\u1D57\u02B0, 50\u1D57\u02B0, 75\u1D57\u02B0, 90\u1D57\u02B0)"
    )

    ax[0].set_title(title0, ha="left", loc="left")
    ax[1].set_title(title1, ha="left", loc="left")

    f.tight_layout(w_pad=2.0)
    plt.subplots_adjust(top=0.8, bottom=0.3)
    if pu.save_fig:
        f.savefig(
            pu.si_path
            / f"Figure S7 {pu.plot_numbers[2*icase]}-{pu.plot_numbers[2*icase+1]}.pdf"
        )